In [2]:
import numpy as np
import pandas as pd
import glob
import os
import sys
from pandas import ExcelWriter

In [ ]:

nv1=pd.read_csv('conjFDR_0.05_GGE_vs_COG_lead.csv',sep='\t')

nv1.head()

gw1=pd.read_csv('gwascatalog.txt',sep='\t')
gw1.head()

gw1.columns

snp1=pd.read_csv('conjFDR_0.05_GGE_vs_COG_snps.csv',sep='\t')
snp1.head()

db1=pd.read_csv('novelty_ep4.csv',sep='\t')
db1.head()

nov='epilep'

gw2=gw1[gw1['Trait'].str.contains(fr'{nov}(?!$)')]
gw2.head()

gw1.Trait.value_counts()

#snp1['Novel_in_GWAScatalog'] = snp1.groupby('locusnum')['CAND_SNP'].apply(lambda s: s.isin(gw2['snp']))

new_c=snp1.groupby('locusnum')['CAND_SNP'].apply(lambda s: s.isin(gw2['snp']))

snp1['Novel_in_GWAScatalog']=new_c.reset_index(level=0, drop=True)

snp1.head()

nv1=nv1.merge(snp1.groupby('locusnum')['Novel_in_GWAScatalog'].any(), left_on='locusnum', right_index=True, how='left')

nv1.Novel_in_GWAScatalog=nv1.Novel_in_GWAScatalog.map({True: 'No', False: 'Yes'})

nv1['MinBP']=nv1['MinBP'].astype(int)
nv1['MaxBP']=nv1['MaxBP'].astype(int)
nv1.CHR=nv1.CHR.astype(int)
db1=db1[db1.chromosome.astype('str').str.isnumeric()]
db1.chromosome=db1.chromosome.astype(int)
db1['min_bp']=db1['min_bp'].astype(int)
db1['max_bp']=db1['max_bp'].astype(int)
a = nv1['MinBP'].to_numpy()
b = nv1['MaxBP'].to_numpy()
c1= nv1.CHR.to_numpy()
c2= db1.chromosome.to_numpy()[:,None]
# Raise the columns in db1 by a dimension to enable numpy's broadcasting
c = db1['min_bp'].to_numpy()[:, None]
d = db1['max_bp'].to_numpy()[:, None]

result = ((c1 == c2) & (a >= c) & (a <= d)) | ((c1 == c2) & (b >= c) & (b <= d)) | ((c1 == c2) & (a <= c) & (b >= d))

nv1['Novel_in_Database'] = np.where(result.any(axis=0), 'No', 'Yes')

trait1='GGE'
trait2='COG'

nv1[f'Novel_in_{trait1}']=np.where(((nv1.Novel_in_GWAScatalog=="Yes") & (nv1.Novel_in_Database=='Yes')),'Yes','No')

nv1=nv1.drop(['Novel_in_GWAScatalog', 'Novel_in_Database'], axis = 1)

nv1.to_csv(f'conjFDR_0.05_{trait1}_vs_{trait2}_novelty.csv',index=False,sep='\t')



In [35]:
nv1=pd.read_csv('conjFDR_0.05_GGE_vs_COG_lead.csv',sep='\t')

nv1.head()

gw1=pd.read_csv('gwascatalog.txt',sep='\t')
gw1.head()

gw1.columns

snp1=pd.read_csv('conjFDR_0.05_GGE_vs_COG_snps.csv',sep='\t')
snp1.head()

db1=pd.read_csv('novelty_checking_database_cog.csv',sep='\t')
db1.head()

nov='cognit'

gw2=gw1[gw1['Trait'].str.contains(fr'{nov}(?!$)')]
gw2.head()

gw1.Trait.value_counts()

#snp1['Novel_in_GWAScatalog'] = snp1.groupby('locusnum')['CAND_SNP'].apply(lambda s: s.isin(gw2['snp']))

new_c=snp1.groupby('locusnum')['CAND_SNP'].apply(lambda s: s.isin(gw2['snp']))

snp1['Novel_in_GWAScatalog']=new_c.reset_index(level=0, drop=True)

snp1.head()

nv1=nv1.merge(snp1.groupby('locusnum')['Novel_in_GWAScatalog'].any(), left_on='locusnum', right_index=True, how='left')

nv1.Novel_in_GWAScatalog=nv1.Novel_in_GWAScatalog.map({True: 'No', False: 'Yes'})

nv1['MinBP']=nv1['MinBP'].astype(int)
nv1['MaxBP']=nv1['MaxBP'].astype(int)
nv1.CHR=nv1.CHR.astype(int)
db1=db1[db1.chromosome.astype('str').str.isnumeric()]
db1.chromosome=db1.chromosome.astype(int)
db1['min_bp']=db1['min_bp'].astype(int)
db1['max_bp']=db1['max_bp'].astype(int)
a = nv1['MinBP'].to_numpy()
b = nv1['MaxBP'].to_numpy()
c1= nv1.CHR.to_numpy()
c2= db1.chromosome.to_numpy()[:,None]
# Raise the columns in db1 by a dimension to enable numpy's broadcasting
c = db1['min_bp'].to_numpy()[:, None]
d = db1['max_bp'].to_numpy()[:, None]

result = ((c1 == c2) & (a >= c) & (a <= d)) | ((c1 == c2) & (b >= c) & (b <= d)) | ((c1 == c2) & (a <= c) & (b >= d))

nv1['Novel_in_Database'] = np.where(result.any(axis=0), 'No', 'Yes')

trait1='COG'
trait2='GGE'

nv1[f'Novel_in_{trait1}']=np.where(((nv1.Novel_in_GWAScatalog=="Yes") & (nv1.Novel_in_Database=='Yes')),'Yes','No')

nv1=nv1.drop(['Novel_in_GWAScatalog', 'Novel_in_Database'], axis = 1)

nv1.to_csv(f'conjFDR_0.05_{trait1}_vs_{trait2}_novelty.csv',index=False,sep='\t')


In [2]:
def novelty_check(lead,gwas,snp,db,nov,trait1,trait2):
    nv1=pd.read_csv(lead,sep='\t')
    nv1.head()
    gw1=pd.read_csv(gwas,sep='\t')
    gw1.head()

    gw1.columns

    snp1=pd.read_csv(snp,sep='\t')
    snp1.head()

    db1=pd.read_csv(db,sep='\t')
    db1.head()

    #nov='epilep'

    gw2=gw1[gw1['Trait'].str.contains(fr'{nov}(?!$)')]
    gw2.head()

    gw1.Trait.value_counts()

    #snp1['Novel_in_GWAScatalog'] = snp1.groupby('locusnum')['CAND_SNP'].apply(lambda s: s.isin(gw2['snp']))

    new_col=snp1.groupby('locusnum')['CAND_SNP'].apply(lambda s: s.isin(gw2['snp']))

    snp1['Novel_in_GWAScatalog']=new_col.reset_index(level=0, drop=True)

    snp1.head()

    nv1=nv1.merge(snp1.groupby('locusnum')['Novel_in_GWAScatalog'].any(), left_on='locusnum', right_index=True, how='left')

    nv1.Novel_in_GWAScatalog=nv1.Novel_in_GWAScatalog.map({True: 'No', False: 'Yes'})

    nv1['MinBP']=nv1['MinBP'].astype(int)
    nv1['MaxBP']=nv1['MaxBP'].astype(int)
    nv1.CHR=nv1.CHR.astype(int)
    db1=db1[db1.chromosome.astype('str').str.isnumeric()]
    db1.chromosome=db1.chromosome.astype(int)
    db1['min_bp']=db1['min_bp'].astype(int)
    db1['max_bp']=db1['max_bp'].astype(int)
    a = nv1['MinBP'].to_numpy()
    b = nv1['MaxBP'].to_numpy()
    c1= nv1.CHR.to_numpy()
    c2= db1.chromosome.to_numpy()[:,None]
    # Raise the columns in db1 by a dimension to enable numpy's broadcasting
    c = db1['min_bp'].to_numpy()[:, None]
    d = db1['max_bp'].to_numpy()[:, None]

    result = ((c1 == c2) & (a >= c) & (a <= d)) | ((c1 == c2) & (b >= c) & (b <= d)) | ((c1 == c2) & (a <= c) & (b >= d))

    nv1['Novel_in_Database'] = np.where(result.any(axis=0), 'No', 'Yes')

    #trait1='GGE'
    #trait2='COG'

    nv1[f'Novel_in_{trait1}']=np.where(((nv1.Novel_in_GWAScatalog=="Yes") & (nv1.Novel_in_Database=='Yes')),'Yes','No')

    nv1=nv1.drop(['Novel_in_GWAScatalog', 'Novel_in_Database'], axis = 1)

    nv1.to_csv(f'conjFDR_0.05_{trait1}_vs_{trait2}_novelty.csv',index=False,sep='\t')


In [3]:
os.getcwd()

'/Users/zillurrahman/Desktop/zillur/work/coauthorships/novelty_for_naz'

In [6]:
novelty_check('zillur/all/conjFDR_0.05_ALL_vs_COG_lead.csv','zillur/all/gwascatalog.txt','zillur/all/conjFDR_0.05_ALL_vs_COG_snps.csv',
             'zillur/all/novelty_ep4.csv','epilep','All','COG')

In [8]:
novelty_check('zillur/all/conjFDR_0.05_ALL_vs_COG_lead.csv','zillur/all/gwascatalog.txt','zillur/all/conjFDR_0.05_ALL_vs_COG_snps.csv',
             'zillur/all/novelty_checking_database_cog.csv','cognit','COG','All')

In [9]:
novelty_check('zillur/cae/conjFDR_0.05_CAE_vs_COG_lead.csv','zillur/cae/gwascatalog.txt','zillur/cae/conjFDR_0.05_CAE_vs_COG_snps.csv',
             'zillur/cae/novelty_ep4.csv','epilep','CAE','COG')
novelty_check('zillur/cae/conjFDR_0.05_CAE_vs_COG_lead.csv','zillur/cae/gwascatalog.txt','zillur/cae/conjFDR_0.05_CAE_vs_COG_snps.csv',
             'zillur/cae/novelty_checking_database_cog.csv','cognit','COG','CAE')

In [10]:
novelty_check('zillur/jme/conjFDR_0.05_JME_vs_COG_lead.csv','zillur/jme/gwascatalog.txt','zillur/jme/conjFDR_0.05_JME_vs_COG_snps.csv',
             'zillur/jme/novelty_ep4.csv','epilep','JME','COG')
novelty_check('zillur/jme/conjFDR_0.05_JME_vs_COG_lead.csv','zillur/jme/gwascatalog.txt','zillur/jme/conjFDR_0.05_JME_vs_COG_snps.csv',
             'zillur/jme/novelty_checking_database_cog.csv','cognit','COG','JME')

In [3]:
hsk1 = pd.read_excel('../../husk_ment/prime/kcnq1_magma_report_v1_dec24.xlsx')
hsk1.head()

,Data_set,Trait,Description,N,Ncase,Ncontrol,Covariates,Population,GENE,CHR,START,STOP,NSNPS,NPARAM,N.1,ZSTAT,P,P_adjusted
0,BCX,WBC,Publicely Available,NaN,NaN,NaN,NaN,NaN,KCNQ1,11.0,2466221.0,2870340.0,1490.0,245.0,518638.0,4.21900,1.230000e-01,3.005660e-04
1,CTG,INTELLIGENCE,Publicely Available,NaN,NaN,NaN,NaN,NaN,KCNQ1,11.0,2466221.0,2870340.0,1196.0,187.0,57547.0,0.59066,2.773700e-01,9.707950e-01
2,CTG,INSOMNIA,Publicely Available,NaN,NaN,NaN,NaN,NaN,KCNQ1,11.0,2466221.0,2870340.0,944.0,156.0,382837.0,0.15140,4.398300e-01,9.989800e-01
3,DIAMANTE,T2DM,Publicely Available,NaN,NaN,NaN,NaN,NaN,KCNQ1,11.0,2466221.0,2870340.0,1144.0,181.0,293100.0,14.18000,6.080000e-42,2.980000e-38
4,GSCAN,SMOKE,Publicely Available,NaN,NaN,NaN,NaN,NaN,KCNQ1,11.0,2466221.0,2870340.0,1388.0,234.0,227978.0,-0.35123,6.372900e-01,9.989800e-01


In [4]:
from statsmodels.stats.multitest import multipletests

In [7]:
hsk1['P'] = hsk1['P'].astype(float)

In [8]:
_, p_adjusted, _, _ = multipletests(hsk1['P'], method='fdr_bh')

In [9]:
hsk1['P_adjusted'] = p_adjusted
hsk1.head()

,Data_set,Trait,Description,N,Ncase,Ncontrol,Covariates,Population,GENE,CHR,START,STOP,NSNPS,NPARAM,N.1,ZSTAT,P,P_adjusted
0,BCX,WBC,Publicely Available,NaN,NaN,NaN,NaN,NaN,KCNQ1,11.0,2466221.0,2870340.0,1490.0,245.0,518638.0,4.21900,1.230000e-01,NaN
1,CTG,INTELLIGENCE,Publicely Available,NaN,NaN,NaN,NaN,NaN,KCNQ1,11.0,2466221.0,2870340.0,1196.0,187.0,57547.0,0.59066,2.773700e-01,NaN
2,CTG,INSOMNIA,Publicely Available,NaN,NaN,NaN,NaN,NaN,KCNQ1,11.0,2466221.0,2870340.0,944.0,156.0,382837.0,0.15140,4.398300e-01,NaN
3,DIAMANTE,T2DM,Publicely Available,NaN,NaN,NaN,NaN,NaN,KCNQ1,11.0,2466221.0,2870340.0,1144.0,181.0,293100.0,14.18000,6.080000e-42,NaN
4,GSCAN,SMOKE,Publicely Available,NaN,NaN,NaN,NaN,NaN,KCNQ1,11.0,2466221.0,2870340.0,1388.0,234.0,227978.0,-0.35123,6.372900e-01,NaN


In [10]:
hsk1.shape

(77, 18)

In [12]:
hsk1.columns

Index(['Data_set', 'Trait', 'Description', 'N', 'Ncase', 'Ncontrol',
       'Covariates', 'Population', 'GENE', 'CHR', 'START', 'STOP', 'NSNPS',
       'NPARAM', 'N.1', 'ZSTAT', 'P', 'P_adjusted'],
      dtype='object')

In [13]:
hsk2 = hsk1.drop('P_adjusted',axis=1)
hsk2.head()

,Data_set,Trait,Description,N,Ncase,Ncontrol,Covariates,Population,GENE,CHR,START,STOP,NSNPS,NPARAM,N.1,ZSTAT,P
0,BCX,WBC,Publicely Available,NaN,NaN,NaN,NaN,NaN,KCNQ1,11.0,2466221.0,2870340.0,1490.0,245.0,518638.0,4.21900,1.230000e-01
1,CTG,INTELLIGENCE,Publicely Available,NaN,NaN,NaN,NaN,NaN,KCNQ1,11.0,2466221.0,2870340.0,1196.0,187.0,57547.0,0.59066,2.773700e-01
2,CTG,INSOMNIA,Publicely Available,NaN,NaN,NaN,NaN,NaN,KCNQ1,11.0,2466221.0,2870340.0,944.0,156.0,382837.0,0.15140,4.398300e-01
3,DIAMANTE,T2DM,Publicely Available,NaN,NaN,NaN,NaN,NaN,KCNQ1,11.0,2466221.0,2870340.0,1144.0,181.0,293100.0,14.18000,6.080000e-42
4,GSCAN,SMOKE,Publicely Available,NaN,NaN,NaN,NaN,NaN,KCNQ1,11.0,2466221.0,2870340.0,1388.0,234.0,227978.0,-0.35123,6.372900e-01


In [14]:
hsk2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Data_set     67 non-null     object 
 1   Trait        58 non-null     object 
 2   Description  58 non-null     object 
 3   N            36 non-null     float64
 4   Ncase        36 non-null     object 
 5   Ncontrol     36 non-null     object 
 6   Covariates   36 non-null     object 
 7   Population   36 non-null     object 
 8   GENE         58 non-null     object 
 9   CHR          58 non-null     float64
 10  START        58 non-null     float64
 11  STOP         58 non-null     float64
 12  NSNPS        58 non-null     float64
 13  NPARAM       58 non-null     float64
 14  N.1          58 non-null     float64
 15  ZSTAT        58 non-null     float64
 16  P            58 non-null     float64
dtypes: float64(9), object(8)
memory usage: 10.4+ KB


In [15]:
_, p_adjusted, _, _ = multipletests(hsk2['P'], method='fdr_bh')

In [16]:
p_adjusted

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [19]:
87600*2 + 63000

238200

In [20]:
21242*11.05

234724.1

In [21]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 6.5 MB/s eta 0:00:00a 0:00:01


In [22]:
import fitz

In [40]:
def pdf_to_png(pdf_path, output_folder,name, zoom=2.0, rotation=0):
    # Open the PDF file
    pdf_document = fitz.open(pdf_path)
    
    # Iterate over each page
    for page_number in range(len(pdf_document)):
        # Select the page
        page = pdf_document.load_page(page_number)
        
        # Create the transform for scaling
        matrix = fitz.Matrix(zoom, zoom)
        
        # Render the page to a pixmap
        pixmap = page.get_pixmap(matrix=matrix)
        
        # Define the output path for the PNG file
        output_path = f"{output_folder}/{name}_photo_{page_number + 1}.jpeg"
        
        # Save the image as a PNG file
        pixmap.save(output_path)
        
    print("PDF has been converted to PNG and saved in the specified folder.")


In [37]:
pdf_to_png('/Users/zillurrahman/Documents/personal/zillur-new-passport-2022-2032.pdf',
           '/Users/zillurrahman/Documents/personal','zillur')

PDF has been converted to PNG and saved in the specified folder.


In [38]:
pdf_to_png('/Users/zillurrahman/Documents/personal/all_documents_from_oslo/nasrin_akter/nupa_passport_visa.pdf',
           '/Users/zillurrahman/Documents/personal','nupa')

PDF has been converted to PNG and saved in the specified folder.


In [39]:
pdf_to_png('/Users/zillurrahman/Documents/renewal_residence/sannidho/sannidho_passport.pdf',
           '/Users/zillurrahman/Documents/personal','sannidho')

PDF has been converted to PNG and saved in the specified folder.


In [41]:
pdf_to_png('/Users/zillurrahman/Documents/personal/zillur_photo_pp.pdf',
           '/Users/zillurrahman/Documents/personal','zillur')

PDF has been converted to PNG and saved in the specified folder.


In [42]:
pdf_to_png('/Users/zillurrahman/Documents/personal/nupa_photo_pp.pdf',
           '/Users/zillurrahman/Documents/personal','nupa')

PDF has been converted to PNG and saved in the specified folder.


In [6]:
!pip install gdown

In [7]:
import gdown

In [9]:
from io import BytesIO

In [8]:
link1 = 'https://docs.google.com/spreadsheets/d/1PsAcFRlG0x_JZOknWLQ_GQiCyR_p5ZDiaR8TqR2NwoQ/edit?usp=sharing'

In [13]:
link2 = f'https://drive.google.com/uc?export=download&id=1PsAcFRlG0x_JZOknWLQ_GQiCyR_p5ZDiaR8TqR2NwoQ'

In [14]:
db1 = pd.read_excel(BytesIO(gdown.download(link2, quiet=True)),engine='openpyxl')

/Users/zillurrahman/miniconda3/lib/python3.10/site-packages/gdown/download.py:33: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs4.BeautifulSoup(line, features="html.parser")


FileURLRetrievalError: Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?export=download&id=1PsAcFRlG0x_JZOknWLQ_GQiCyR_p5ZDiaR8TqR2NwoQ

but Gdown can't. Please check connections and permissions.

In [15]:
os.getcwd()

'/Users/zillurrahman/Desktop/zillur/work/body-brain/borge'

In [26]:
db1 = pd.read_excel('novelty_checking_database.xlsx',engine='openpyxl',sheet_name=None)

In [27]:
aud_db1 = db1.get('aud')
aud_db1.head()

,Alcohol use disorder,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Date of last update,04.05.2023,KEY,study = abbreviated study name as defined at t...,NaN,NaN,NaN,NaN
1,Description of search completed,NaN,NaN,method = GWAS/condFDR/conjFDR/MTAG/STAR/other,NaN,NaN,NaN,NaN
2,Last update by,Børge,NaN,NaN,NaN,NaN,NaN,NaN
3,Search terms for literature search,Alcohol use disorder + genetic loci,NaN,NaN,NaN,NaN,NaN,NaN
4,List of studies included (author year journal ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
aud_db2=aud_db1.iloc[9:].reset_index(drop=True)
aud_db2.head()

,Alcohol use disorder,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,chromosome,min_bp,max_bp,lead_SNP,study,method,ethnicity,Pos
1,1,NaN,NaN,rs11210887,Xu2023,MTAG,EUR,44076019
2,1,NaN,NaN,rs2310819,Xu2023,MTAG,EUR,66440096
3,1,NaN,NaN,rs11210229,Xu2023,MTAG,EUR,73860028
4,1,NaN,NaN,rs1526480,Xu2023,MTAG,EUR,91209986


In [32]:
new_column_names1 = aud_db2.iloc[0]
new_column_names1

Alcohol use disorder    chromosome
Unnamed: 1                  min_bp
Unnamed: 2                  max_bp
Unnamed: 3                lead_SNP
Unnamed: 4                   study
Unnamed: 5                  method
Unnamed: 6               ethnicity
Unnamed: 7                     Pos
Name: 0, dtype: object

In [33]:
aud_db3 = aud_db2[1:].reset_index(drop=True)
aud_db3.head()

,Alcohol use disorder,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,1,NaN,NaN,rs11210887,Xu2023,MTAG,EUR,44076019
1,1,NaN,NaN,rs2310819,Xu2023,MTAG,EUR,66440096
2,1,NaN,NaN,rs11210229,Xu2023,MTAG,EUR,73860028
3,1,NaN,NaN,rs1526480,Xu2023,MTAG,EUR,91209986
4,1,NaN,NaN,rs3897854,Xu2023,MTAG,EUR,97918306


In [34]:
aud_db3.columns = new_column_names1
aud_db3.head()

,chromosome,min_bp,max_bp,lead_SNP,study,method,ethnicity,Pos
0,1,NaN,NaN,rs11210887,Xu2023,MTAG,EUR,44076019
1,1,NaN,NaN,rs2310819,Xu2023,MTAG,EUR,66440096
2,1,NaN,NaN,rs11210229,Xu2023,MTAG,EUR,73860028
3,1,NaN,NaN,rs1526480,Xu2023,MTAG,EUR,91209986
4,1,NaN,NaN,rs3897854,Xu2023,MTAG,EUR,97918306


In [35]:
cud_db1 = db1.get('cud')
cud_db1.head()

,Problematic opioid use,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Date of last update,04.05.2023,KEY,study = abbreviated study name as defined at t...,NaN,NaN,NaN,NaN
1,Description of search completed,NaN,NaN,method = GWAS/condFDR/conjFDR/MTAG/STAR/other,NaN,NaN,NaN,NaN
2,Last update by,Børge,NaN,NaN,NaN,NaN,NaN,NaN
3,Search terms for literature search,Cannabis use disorder + genetic loci,NaN,NaN,NaN,NaN,EURA = European american,NaN
4,List of studies included (author year journal ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
cud_db2=cud_db1.iloc[8:].reset_index(drop=True)
cud_db2.head()

,Problematic opioid use,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,chromosome,min_bp,max_bp,lead_SNP,study,method,ethnicity,Pos
1,1,NaN,NaN,rs11210887,Xu2023,MTAG,EUR,44076019
2,1,NaN,NaN,rs9436444,Xu2023,MTAG,EUR,50589798
3,1,66334537,66385385,rs1937460,Cheng2023,conjFDR,EUR,66334537
4,1,NaN,NaN,rs2310819,Xu2023,MTAG,EUR,66440096


In [38]:
new_column_names2 = cud_db2.iloc[0]
new_column_names2

Problematic opioid use    chromosome
Unnamed: 1                    min_bp
Unnamed: 2                    max_bp
Unnamed: 3                  lead_SNP
Unnamed: 4                     study
Unnamed: 5                    method
Unnamed: 6                 ethnicity
Unnamed: 7                       Pos
Name: 0, dtype: object

In [39]:
cud_db3 = cud_db2[1:].reset_index(drop=True)
cud_db3.head()

,Problematic opioid use,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,1,NaN,NaN,rs11210887,Xu2023,MTAG,EUR,44076019
1,1,NaN,NaN,rs9436444,Xu2023,MTAG,EUR,50589798
2,1,66334537,66385385,rs1937460,Cheng2023,conjFDR,EUR,66334537
3,1,NaN,NaN,rs2310819,Xu2023,MTAG,EUR,66440096
4,1,NaN,NaN,rs4593766,Xu2023,MTAG,EUR,73773043


In [40]:
cud_db3.columns = new_column_names2
cud_db3.head()

,chromosome,min_bp,max_bp,lead_SNP,study,method,ethnicity,Pos
0,1,NaN,NaN,rs11210887,Xu2023,MTAG,EUR,44076019
1,1,NaN,NaN,rs9436444,Xu2023,MTAG,EUR,50589798
2,1,66334537,66385385,rs1937460,Cheng2023,conjFDR,EUR,66334537
3,1,NaN,NaN,rs2310819,Xu2023,MTAG,EUR,66440096
4,1,NaN,NaN,rs4593766,Xu2023,MTAG,EUR,73773043


In [41]:
oud_db1 = db1.get('oud')
oud_db1.head()

,Opioid use disorder,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Date of last update,03.05.2023,KEY,study = abbreviated study name as defined at t...,NaN,NaN,NaN,NaN,NaN
1,Description of search completed,NaN,NaN,method = GWAS/condFDR/conjFDR/MTAG/STAR/other,NaN,NaN,NaN,NaN,NaN
2,Last update by,Børge,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Search terms for literature search,Opioid use disorder + genetic loci,NaN,NaN,NaN,NaN,EURA = European american,NaN,NaN
4,List of studies included (author year journal ...,NaN,NaN,NaN,NaN,NaN,AFRA = African american,NaN,NaN


In [43]:
oud_db2=oud_db1.iloc[10:].reset_index(drop=True)
oud_db2.head()

,Opioid use disorder,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,chromosome,min_bp,max_bp,lead_SNP,study,method,ethnicity,Pos,Association with other substance use trait
1,1,27989020,29989020,rs6667501,Deak2021,MTAG,EUR,28989020,NaN
2,1,55522425,55708610,rs67237321,Holen2023,condFDR,EUR,55686228,NaN
3,1,65434743,67434743,rs7519259,Deak2021,MTAG,EUR,66434743,"Smoking, Alcohol (consumption and problematic ..."
4,1,72860028,74860028,rs11210229,Xu2022,MTAG,EUR,73860028,"Smoking, Alcohol (consumption, problematic use..."


In [44]:
new_column_names3 = oud_db2.iloc[0]
new_column_names3

Opioid use disorder                                    chromosome
Unnamed: 1                                                 min_bp
Unnamed: 2                                                 max_bp
Unnamed: 3                                               lead_SNP
Unnamed: 4                                                  study
Unnamed: 5                                                 method
Unnamed: 6                                              ethnicity
Unnamed: 7                                                    Pos
Unnamed: 8             Association with other substance use trait
Name: 0, dtype: object

In [45]:
oud_db3 = oud_db2[1:].reset_index(drop=True)
oud_db3.head()

,Opioid use disorder,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,1,27989020,29989020,rs6667501,Deak2021,MTAG,EUR,28989020,NaN
1,1,55522425,55708610,rs67237321,Holen2023,condFDR,EUR,55686228,NaN
2,1,65434743,67434743,rs7519259,Deak2021,MTAG,EUR,66434743,"Smoking, Alcohol (consumption and problematic ..."
3,1,72860028,74860028,rs11210229,Xu2022,MTAG,EUR,73860028,"Smoking, Alcohol (consumption, problematic use..."
4,1,90208451,92208451,rs2166171,Deak2021,MTAG,EUR,91208451,Smoking


In [46]:
oud_db3.columns = new_column_names3
oud_db3.head()

,chromosome,min_bp,max_bp,lead_SNP,study,method,ethnicity,Pos,Association with other substance use trait
0,1,27989020,29989020,rs6667501,Deak2021,MTAG,EUR,28989020,NaN
1,1,55522425,55708610,rs67237321,Holen2023,condFDR,EUR,55686228,NaN
2,1,65434743,67434743,rs7519259,Deak2021,MTAG,EUR,66434743,"Smoking, Alcohol (consumption and problematic ..."
3,1,72860028,74860028,rs11210229,Xu2022,MTAG,EUR,73860028,"Smoking, Alcohol (consumption, problematic use..."
4,1,90208451,92208451,rs2166171,Deak2021,MTAG,EUR,91208451,Smoking


In [48]:
aud_db3.head()

,chromosome,min_bp,max_bp,lead_SNP,study,method,ethnicity,Pos
0,1,NaN,NaN,rs11210887,Xu2023,MTAG,EUR,44076019
1,1,NaN,NaN,rs2310819,Xu2023,MTAG,EUR,66440096
2,1,NaN,NaN,rs11210229,Xu2023,MTAG,EUR,73860028
3,1,NaN,NaN,rs1526480,Xu2023,MTAG,EUR,91209986
4,1,NaN,NaN,rs3897854,Xu2023,MTAG,EUR,97918306


In [49]:
cud_db3.head()

,chromosome,min_bp,max_bp,lead_SNP,study,method,ethnicity,Pos
0,1,NaN,NaN,rs11210887,Xu2023,MTAG,EUR,44076019
1,1,NaN,NaN,rs9436444,Xu2023,MTAG,EUR,50589798
2,1,66334537,66385385,rs1937460,Cheng2023,conjFDR,EUR,66334537
3,1,NaN,NaN,rs2310819,Xu2023,MTAG,EUR,66440096
4,1,NaN,NaN,rs4593766,Xu2023,MTAG,EUR,73773043


In [50]:
condfdr1 = pd.read_excel('fdr_out/condfdr_lead_snp2gene_effects_fin1_novelty.xlsx',sheet_name=None)
condfdr1.keys()

dict_keys(['CUD|OUD', 'CUD|AUD', 'OUD|CUD', 'OUD|AUD', 'AUD|CUD', 'AUD|OUD'])

In [76]:
w2 = ExcelWriter('fdr_out/condfdr_lead_snp2gene_effects_fin1_novelty3.xlsx')
for df in condfdr1.keys():
    df1 = condfdr1.get(df)
    tr1 = df.split('|')[0]
    tr2 = df.split('|')[1]
    if tr1 == 'AUD':
        db1 = aud_db3
    elif tr1 == 'CUD':
        db1 = cud_db3
    else:
        db1 = oud_db3
    if tr2 == 'AUD':
        db2 = aud_db3
    elif tr2 == 'CUD':
        db2 = cud_db3
    else:
        db2 = oud_db3
    
    #Process query and db for loci search
    df2 = df1.iloc[:, :-4]
    df2['Novel_leadsnp_db1'] = np.where(df2['LEAD_SNP'].isin(db1['lead_SNP']), 'No', 'Yes')
    df2['Novel_leadsnp_db2'] = np.where(df2['LEAD_SNP'].isin(db2['lead_SNP']), 'No', 'Yes')
    df2[['CHR', 'MinBP', 'MaxBP']] = df2[['CHR', 'MinBP', 'MaxBP']].apply(pd.to_numeric, errors='coerce')
    db1[['chromosome', 'min_bp', 'max_bp']] = db1[['chromosome', 'min_bp', 'max_bp']].apply(pd.to_numeric, errors='coerce')
    db2[['chromosome', 'min_bp', 'max_bp']] = db2[['chromosome', 'min_bp', 'max_bp']].apply(pd.to_numeric, errors='coerce')
    db1 = db1[['chromosome', 'min_bp', 'max_bp']]
    db2 = db2[['chromosome', 'min_bp', 'max_bp']]
    db1.dropna(inplace=True)
    db2.dropna(inplace=True)

    # Process for db_tr1
    if not db1.iloc[:, 1].isna().any() and not db1.iloc[:, 2].isna().any():
        c1 = df2.iloc[:, 1].to_numpy()[:, None]  # Column 2 of dt2
        c2 = db1.iloc[:, 0].to_numpy()[:, None]  # Column 2 of db_tr1
        a = df2.iloc[:, 4].to_numpy()[:, None]  # Column 4 of dt2
        b = df2.iloc[:, 5].to_numpy()[:, None] # Column 5 of dt2
        d1 = db1.iloc[:, 1].to_numpy()[:, None]  # Column 3 of db_tr1
        d2 = db1.iloc[:, 2].to_numpy()[:, None]  # Column 4 of db_tr1

        result1 = (
            ((c1 == c2.T) & (a >= d1.T) & (a <= d2.T)) |
            ((c1 == c2.T) & (b >= d1.T) & (b <= d2.T)) |
            ((c1 == c2.T) & (a <= d1.T) & (b >= d2.T)) |
            ((c1 == c2.T) & (a >= d1.T) & (b <= d2.T))
        )
        df2['Novel_loci_in_db1'] = np.where(result1.any(axis=1), 'No', 'Yes')
    
    if not db2.iloc[:, 1].isna().any() and not db2.iloc[:, 2].isna().any():
        c1 = df2.iloc[:, 1].to_numpy()[:, None]  # Column 2 of dt2
        c2 = db2.iloc[:, 0].to_numpy()[:, None]  # Column 2 of db_tr1
        a = df2.iloc[:, 4].to_numpy()[:, None]  # Column 4 of dt2
        b = df2.iloc[:, 5].to_numpy()[:, None] # Column 5 of dt2
        d1 = db2.iloc[:, 1].to_numpy()[:, None]  # Column 3 of db_tr1
        d2 = db2.iloc[:, 2].to_numpy()[:, None]  # Column 4 of db_tr1

        result2 = (
            ((c1 == c2.T) & (a >= d1.T) & (a <= d2.T)) |
            ((c1 == c2.T) & (b >= d1.T) & (b <= d2.T)) |
            ((c1 == c2.T) & (a <= d1.T) & (b >= d2.T)) |
            ((c1 == c2.T) & (a >= d1.T) & (b <= d2.T))
        )
        df2['Novel_loci_in_db2'] = np.where(result2.any(axis=1), 'No', 'Yes')


    df2['Novel_loci'] = np.where((df2['Novel_leadsnp_db1'] == 'Yes') & (df2['Novel_leadsnp_db2'] == 'Yes') & 
                                (df2['Novel_loci_in_db1'] == 'Yes') & (df2['Novel_loci_in_db2'] == 'Yes'),'Yes','No' )
    df2.to_excel(w2,sheet_name=f'{tr1}|{tr2}',index=False)
    print(tr1,db1.shape,tr2,db2.shape,df2.head())
w2.close()

CUD (24, 3) OUD (31, 3)    locusnum  CHR    LEAD_SNP    LEAD_BP      MinBP      MaxBP       FDR  \
0         1    2  rs12616949   22594027   22549441   22606275  0.006116   
1         2    5  rs12513581   60533207   60484179   60614879  0.008724   
2         3    6    rs975303   19028788   19023726   19191459  0.001397   
3         4    6   rs3734540   26463321   26393021   26581168  0.001896   
4         5    7   rs1989903  114137940  114015707  114287116  0.001041   

   is_locus_lead A1 A2  ...  PVAL_in_OUD             Gene Approved symbol  \
0           True  A  G  ...     0.003622              NaN             NaN   
1           True  C  T  ...     0.002643  ENSG00000164181          ELOVL7   
2           True  G  A  ...     0.000019              NaN             NaN   
3           True  C  A  ...     0.002091  ENSG00000182952           HMGN4   
4           True  A  G  ...     0.082183  ENSG00000128573           FOXP2   

                                      Approved name  Overall_G

/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2763236127.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db1.dropna(inplace=True)
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2763236127.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db2.dropna(inplace=True)
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2763236127.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db1.d

In [70]:
conjfdr1 = pd.read_excel('fdr_out/conjfdr_lead_snp2gene_effects_fin1_novelty.xlsx',sheet_name=None)
conjfdr1.keys()

dict_keys(['CUD_OUD', 'CUD_AUD', 'AUD_OUD'])

In [72]:
for df in conjfdr1.keys():
    df1 = conjfdr1.get(df)
    tr1 = df.split('_')[0]
    tr2 = df.split('_')[1]
    print(df1.head())

   locusnum  CHR    LEAD_SNP    LEAD_BP      MinBP      MaxBP       FDR  \
0         1    6  rs72828517   19036035   19023726   19191459  0.006021   
1         2   11  rs11214677  113477081  113464248  113481638  0.033113   

   is_locus_lead A1 A2  ...   PVAL_in_CUD  PVAL_in_OUD             Gene  \
0           True  C  T  ...  1.860000e-03     0.000014  ENSG00000137393   
1           True  C  T  ...  4.155000e-07     0.000006  ENSG00000166682   

   Approved symbol                    Approved name Overall_Gene_Score  \
0          RNF144B         ring finger protein 144B           0.000000   
1          TMPRSS5  transmembrane serine protease 5           0.313481   

  Novel_SNP Novel_in_db1  Novel_in_db2  Novel_loci  
0      True          Yes            No       False  
1     False          Yes            No       False  

[2 rows x 23 columns]
   locusnum  CHR     LEAD_SNP    LEAD_BP      MinBP      MaxBP       FDR  \
0         1    1    rs6690101   66414039   66364465   66547212  0.0

In [75]:
w2 = ExcelWriter('fdr_out/conjfdr_lead_snp2gene_effects_fin1_novelty3.xlsx')
for df in conjfdr1.keys():
    df1 = conjfdr1.get(df)
    tr1 = df.split('_')[0]
    tr2 = df.split('_')[1]
    if tr1 == 'AUD':
        db1 = aud_db3
    elif tr1 == 'CUD':
        db1 = cud_db3
    else:
        db1 = oud_db3
    if tr2 == 'AUD':
        db2 = aud_db3
    elif tr2 == 'CUD':
        db2 = cud_db3
    else:
        db2 = oud_db3
    df2 = df1.iloc[:, :-4]
    df2['Novel_leadsnp_db1'] = np.where(df2['LEAD_SNP'].isin(db1['lead_SNP']), 'No', 'Yes')
    df2['Novel_leadsnp_db2'] = np.where(df2['LEAD_SNP'].isin(db2['lead_SNP']), 'No', 'Yes')
    
    #Process db and query for loci search
    df2[['CHR', 'MinBP', 'MaxBP']] = df2[['CHR', 'MinBP', 'MaxBP']].apply(pd.to_numeric, errors='coerce')
    db1[['chromosome', 'min_bp', 'max_bp']] = db1[['chromosome', 'min_bp', 'max_bp']].apply(pd.to_numeric, errors='coerce')
    db2[['chromosome', 'min_bp', 'max_bp']] = db2[['chromosome', 'min_bp', 'max_bp']].apply(pd.to_numeric, errors='coerce')
    db1 = db1[['chromosome', 'min_bp', 'max_bp']]
    db2 = db2[['chromosome', 'min_bp', 'max_bp']]
    db1.dropna(inplace=True)
    db2.dropna(inplace=True)

    # Process for db_tr1
    if not db1.iloc[:, 1].isna().any() and not db1.iloc[:, 2].isna().any():
        c1 = df2.iloc[:, 1].to_numpy()[:, None]  # Column 2 of dt2
        c2 = db1.iloc[:, 0].to_numpy()[:, None]  # Column 2 of db_tr1
        a = df2.iloc[:, 4].to_numpy()[:, None]  # Column 4 of dt2
        b = df2.iloc[:, 5].to_numpy()[:, None] # Column 5 of dt2
        d1 = db1.iloc[:, 1].to_numpy()[:, None]  # Column 3 of db_tr1
        d2 = db1.iloc[:, 2].to_numpy()[:, None]  # Column 4 of db_tr1

        result1 = (
            ((c1 == c2.T) & (a >= d1.T) & (a <= d2.T)) |
            ((c1 == c2.T) & (b >= d1.T) & (b <= d2.T)) |
            ((c1 == c2.T) & (a <= d1.T) & (b >= d2.T)) |
            ((c1 == c2.T) & (a >= d1.T) & (b <= d2.T))
        )
        df2['Novel_loci_in_db1'] = np.where(result1.any(axis=1), 'No', 'Yes')
    
    if not db2.iloc[:, 1].isna().any() and not db2.iloc[:, 2].isna().any():
        c1 = df2.iloc[:, 1].to_numpy()[:, None]  # Column 2 of dt2
        c2 = db2.iloc[:, 0].to_numpy()[:, None]  # Column 2 of db_tr1
        a = df2.iloc[:, 4].to_numpy()[:, None]  # Column 4 of dt2
        b = df2.iloc[:, 5].to_numpy()[:, None] # Column 5 of dt2
        d1 = db2.iloc[:, 1].to_numpy()[:, None]  # Column 3 of db_tr1
        d2 = db2.iloc[:, 2].to_numpy()[:, None]  # Column 4 of db_tr1

        result2 = (
            ((c1 == c2.T) & (a >= d1.T) & (a <= d2.T)) |
            ((c1 == c2.T) & (b >= d1.T) & (b <= d2.T)) |
            ((c1 == c2.T) & (a <= d1.T) & (b >= d2.T)) |
            ((c1 == c2.T) & (a >= d1.T) & (b <= d2.T))
        )
        df2['Novel_loci_in_db2'] = np.where(result2.any(axis=1), 'No', 'Yes')


    df2['Novel_loci'] = np.where((df2['Novel_leadsnp_db1'] == 'Yes') & (df2['Novel_leadsnp_db2'] == 'Yes') & 
                                (df2['Novel_loci_in_db1'] == 'Yes') & (df2['Novel_loci_in_db2'] == 'Yes'),'Yes','No' )
    df2.to_excel(w2,sheet_name=f'{tr1}_{tr2}',index=False)
    print(tr1,db1.shape,tr2,db2.shape,df2.head())
w2.close()

CUD (24, 3) OUD (31, 3)    locusnum  CHR    LEAD_SNP    LEAD_BP      MinBP      MaxBP       FDR  \
0         1    6  rs72828517   19036035   19023726   19191459  0.006021   
1         2   11  rs11214677  113477081  113464248  113481638  0.033113   

   is_locus_lead A1 A2  ...  PVAL_in_OUD             Gene Approved symbol  \
0           True  C  T  ...     0.000014  ENSG00000137393         RNF144B   
1           True  C  T  ...     0.000006  ENSG00000166682         TMPRSS5   

                     Approved name  Overall_Gene_Score Novel_leadsnp_db1  \
0         ring finger protein 144B            0.000000               Yes   
1  transmembrane serine protease 5            0.313481               Yes   

  Novel_leadsnp_db2 Novel_loci_in_db1  Novel_loci_in_db2 Novel_loci  
0               Yes               Yes                Yes        Yes  
1                No               Yes                 No         No  

[2 rows x 24 columns]
CUD (24, 3) AUD (4, 3)    locusnum  CHR     LEAD_SNP    

/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/77464314.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db1.dropna(inplace=True)
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/77464314.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db2.dropna(inplace=True)
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/77464314.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  db1.dropna(

In [77]:
aud_smt1 = pd.read_csv('replication/AUD.AFR.PGC_MVP_UKB.1.190622.metal.processed.tsv.gz',sep='\s+')
aud_smt1.head()

,ID,RSID,CHR,BP,A1,A2,Neff,Z,P,Direction,HetISq,HetChiSq,HetDf,HetPVal
0,9:72398726:A:C,rs144293193,9,72398726,A,C,52192.72,-0.130,0.89690,+--,0.0,1.959,2,0.3755
1,18:3563497:A:G,rs189478541,18,3563497,T,C,48358.95,-0.745,0.45650,?-+,0.0,0.058,1,0.8095
2,2:217147791:A:C,rs6756055,2,217147791,T,G,50799.60,0.387,0.69860,++-,0.0,0.277,2,0.8708
3,5:65691902:A:G,rs549927140,5,65691902,T,C,51761.70,-1.105,0.26920,--+,0.0,0.556,2,0.7572
4,17:38855226:A:C,rs140894156,17,38855226,T,G,48015.20,2.526,0.01155,?+?,0.0,0.000,0,1.0000


In [78]:
aud_smt1.shape

(9903494, 14)

In [79]:
cud_smt1 = pd.read_csv('replication/PGC_CUD_2020_AFR.sumstats.gz',sep='\s+')
cud_smt1.head()

,CHR,POS,RSID,EffectAllele,OtherAllele,Z,P,N,CaseN,ControlN
0,10,101759979,rs138880521,G,A,1.225,0.2204,4442,1505,2937
1,10,101759992,rs11596870,C,T,-0.878,0.3797,6970,2472,4498
2,10,101760107,rs74152907,C,T,-0.696,0.4864,6844,2412,4432
3,10,101760269,rs116820614,G,A,0.408,0.6830,7322,2627,4695
4,10,101760622,rs11190358,A,C,0.483,0.6292,7000,2483,4517


In [80]:
cud_smt1.shape

(12645574, 10)

In [81]:
oud_smt1 = pd.read_csv('replication/OUD_less_stringent.AFR.MVP.DrugAlcDep2021.txt.gz',sep='\s+')
oud_smt1.head()

,SNP_ID,Chromosome,Position,Allele_1,Allele_2,Effect_allele,EAF,Sample_size,OR,SE,Test_statistic,P_value
0,rs7545818,1,100002882,T,G,T,0.646547,88497,0.994339,0.017071,-0.332567,0.739461
1,rs6698430,1,100009669,T,C,T,0.642837,88497,0.993576,0.017037,-0.378256,0.705241
2,rs1416883,1,100010999,G,A,G,0.653375,88497,0.994357,0.016789,-0.337040,0.736087
3,rs1416882,1,100011253,G,A,G,0.643285,88497,0.993754,0.017026,-0.368009,0.712866
4,rs2392043,1,100015373,T,C,T,0.645206,88497,0.994300,0.017101,-0.334269,0.738177


In [82]:
aud_smt1.columns

Index(['ID', 'RSID', 'CHR', 'BP', 'A1', 'A2', 'Neff', 'Z', 'P', 'Direction',
       'HetISq', 'HetChiSq', 'HetDf', 'HetPVal'],
      dtype='object')

In [83]:
cud_smt1.columns

Index(['CHR', 'POS', 'RSID', 'EffectAllele', 'OtherAllele', 'Z', 'P', 'N',
       'CaseN', 'ControlN'],
      dtype='object')

In [84]:
oud_smt1.columns

Index(['SNP_ID', 'Chromosome', 'Position', 'Allele_1', 'Allele_2',
       'Effect_allele', 'EAF', 'Sample_size', 'OR', 'SE', 'Test_statistic',
       'P_value'],
      dtype='object')

In [85]:
cud_smt2 = cud_smt1.rename(columns={'EffectAllele':'A1','OtherAllele':'A2'})
cud_smt2.columns

Index(['CHR', 'POS', 'RSID', 'A1', 'A2', 'Z', 'P', 'N', 'CaseN', 'ControlN'], dtype='object')

In [86]:
oud_smt1['Z'] = oud_smt1['Test_statistic'] / oud_smt1['SE']
oud_smt1.head()

,SNP_ID,Chromosome,Position,Allele_1,Allele_2,Effect_allele,EAF,Sample_size,OR,SE,Test_statistic,P_value,Z
0,rs7545818,1,100002882,T,G,T,0.646547,88497,0.994339,0.017071,-0.332567,0.739461,-19.481629
1,rs6698430,1,100009669,T,C,T,0.642837,88497,0.993576,0.017037,-0.378256,0.705241,-22.202031
2,rs1416883,1,100010999,G,A,G,0.653375,88497,0.994357,0.016789,-0.337040,0.736087,-20.074690
3,rs1416882,1,100011253,G,A,G,0.643285,88497,0.993754,0.017026,-0.368009,0.712866,-21.613896
4,rs2392043,1,100015373,T,C,T,0.645206,88497,0.994300,0.017101,-0.334269,0.738177,-19.546523


In [87]:
oud_smt2 = oud_smt1.rename(columns={'SNP_ID':'RSID','Effect_allele':'A1'})
oud_smt2.columns

Index(['RSID', 'Chromosome', 'Position', 'Allele_1', 'Allele_2', 'A1', 'EAF',
       'Sample_size', 'OR', 'SE', 'Test_statistic', 'P_value', 'Z'],
      dtype='object')

In [98]:
from scipy.stats import binom_test

In [126]:
# Data frame to collect results
results_df = pd.DataFrame(columns=["Trait1", "P_value1","Count_No1", "Count_Total1",
                                   "Trait2","P_value2", "Count_No2","Count_Total2"])

In [129]:
w2 = ExcelWriter('fdr_out/conjfdr_lead_snp2gene_effects_sign_concordance.xlsx')
for df in conjfdr1.keys():
    df1 = conjfdr1.get(df)
    tr1 = df.split('_')[0]
    tr2 = df.split('_')[1]
    if tr1 == 'AUD':
        smt1 = aud_smt1
    elif tr1 == 'CUD':
        smt1 = cud_smt2
    else:
        smt1 = oud_smt2
    if tr2 == 'AUD':
        smt2 = aud_smt1
    elif tr2 == 'CUD':
        smt2 = cud_smt2
    else:
        smt2 = oud_smt2
    df2 = df1.iloc[:, :-4]
    #print(df2.columns,tr1,smt1.columns,tr2,smt2.columns)
    #leadsnp4[f'A1_in_{trait1}'] = leadsnp4['CAND_SNP'].map(sm1.set_index('SNP')['A1'])
    smt1 = smt1.drop_duplicates('RSID')
    smt2 = smt2.drop_duplicates('RSID')
    df2[f'A1_in_{tr1}_AFR'] = df2['LEAD_SNP'].map(smt1.set_index('RSID')['A1'])
    df2[f'Z_in_{tr1}_AFR'] = df2['LEAD_SNP'].map(smt1.set_index('RSID')['Z'])
    df2[f'Z_in_{tr1}_AFR_corrected'] = np.where(df2['A1'] == df2[f'A1_in_{tr1}_AFR'],
                                                df2[f'Z_in_{tr1}_AFR'],df2[f'Z_in_{tr1}_AFR'] * -1)
    mask = df2[f'Z_in_{tr1}_AFR_corrected'].notna()

    # Applying conditional logic using np.where only on rows where Z_in_trait_AFR_corrected is not NaN
    df2.loc[mask, f'sign_concordance_{tr1}_AFR'] = np.where(
        ((df2.loc[mask, f'Z_in_{tr1}'] > 0) & (df2.loc[mask, f'Z_in_{tr1}_AFR_corrected'] > 0)) |
        ((df2.loc[mask, f'Z_in_{tr1}'] < 0) & (df2.loc[mask, f'Z_in_{tr1}_AFR_corrected'] < 0)),
        'Yes','No')
    
    df2[f'A1_in_{tr2}_AFR'] = df2['LEAD_SNP'].map(smt2.set_index('RSID')['A1'])
    df2[f'Z_in_{tr2}_AFR'] = df2['LEAD_SNP'].map(smt2.set_index('RSID')['Z'])
    df2[f'Z_in_{tr2}_AFR_corrected'] = np.where(df2['A1'] == df2[f'A1_in_{tr1}_AFR'],
                                                df2[f'Z_in_{tr1}_AFR'],df2[f'Z_in_{tr1}_AFR'] * -1)
    mask = df2[f'Z_in_{tr2}_AFR_corrected'].notna()

    # Applying conditional logic using np.where only on rows where Z_in_trait_AFR_corrected is not NaN
    df2.loc[mask, f'sign_concordance_{tr2}_AFR'] = np.where(
        ((df2.loc[mask, f'Z_in_{tr2}'] > 0) & (df2.loc[mask, f'Z_in_{tr2}_AFR_corrected'] > 0)) |
        ((df2.loc[mask, f'Z_in_{tr2}'] < 0) & (df2.loc[mask, f'Z_in_{tr2}_AFR_corrected'] < 0)),
        'Yes','No')
    
    # Create a dictionary to store p-values
    p_values = {}

    # Compute the number of "Yes" entries as successes
    x1 = df2.loc[mask, f'sign_concordance_{tr1}_AFR'].eq('No').sum()

    # Compute the total number of trials
    n1 = df2.loc[mask, f'sign_concordance_{tr1}_AFR'].count()

    # Compute the p-value and store it in the dictionary using dynamic key
    key = f'p_value_{tr1}'
    p_values[key] = binom_test(x=x1,n=n1, p=0.5, alternative='less')
    
    # Compute the number of "Yes" entries as successes
    x2 = df2.loc[mask, f'sign_concordance_{tr2}_AFR'].eq('No').sum()

    # Compute the total number of trials
    n2 = df2.loc[mask, f'sign_concordance_{tr2}_AFR'].count()

    # Compute the p-value and store it in the dictionary using dynamic key
    key2 = f'p_value_{tr2}'
    p_values[key2] = binom_test(x=x2,n=n2, p=0.5, alternative='less')
    results_df = results_df.append({
        "Trait1": tr1, 
        "P_value1": p_values[key],
        "Count_No1": x1,
        "Count_Total1": n1,
        "Trait2": tr2, 
        "P_value2": p_values[key2],
        "Count_No2": x2,
        "Count_Total2": n2 
    }, ignore_index=True)
    df2.to_excel(w2,sheet_name=f'{tr1}_{tr2}',index=False)
    results_df.to_excel(w2, sheet_name="binomial_summary", index=False)


    print(df2.head(),tr1,p_values[key],tr2,p_values[key2])
w2.close()

/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2208053712.py:58: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_values[key] = binom_test(x=x1,n=n1, p=0.5, alternative='less')
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2208053712.py:68: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_values[key2] = binom_test(x=x2,n=n2, p=0.5, alternative='less')
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2208053712.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


   locusnum  CHR    LEAD_SNP    LEAD_BP      MinBP      MaxBP       FDR  \
0         1    6  rs72828517   19036035   19023726   19191459  0.006021   
1         2   11  rs11214677  113477081  113464248  113481638  0.033113   

   is_locus_lead A1 A2  ...                    Approved name  \
0           True  C  T  ...         ring finger protein 144B   
1           True  C  T  ...  transmembrane serine protease 5   

   Overall_Gene_Score A1_in_CUD_AFR  Z_in_CUD_AFR  Z_in_CUD_AFR_corrected  \
0            0.000000             T         0.175                  -0.175   
1            0.313481             C         0.285                   0.285   

  sign_concordance_CUD_AFR A1_in_OUD_AFR Z_in_OUD_AFR  Z_in_OUD_AFR_corrected  \
0                      Yes             C   -11.091629                  -0.175   
1                      Yes             T   107.490634                   0.285   

  sign_concordance_OUD_AFR  
0                      Yes  
1                      Yes  

[2 rows x 27 colu

/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2208053712.py:58: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_values[key] = binom_test(x=x1,n=n1, p=0.5, alternative='less')
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2208053712.py:68: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_values[key2] = binom_test(x=x2,n=n2, p=0.5, alternative='less')
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2208053712.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


   locusnum  CHR     LEAD_SNP    LEAD_BP      MinBP      MaxBP       FDR  \
0         1    1    rs6690101   66414039   66364465   66547212  0.003342   
1         2    1    rs1526480   91209986   91189731   91234126  0.007359   
2         3    3   rs61557789   49871201   49609477   49890967  0.026278   
3         4    4  rs190653003  100415029  100395414  100638613  0.015596   
4         5    5    rs1807017   60127724   59842971   60565864  0.042540   

   is_locus_lead A1 A2  ...                       Approved name  \
0           True  T  C  ...                phosphodiesterase 4B   
1           True  C  T  ...                BarH like homeobox 2   
2           True  T  C  ...         RNA binding motif protein 6   
3           True  A  G  ...  chromosome 4 open reading frame 17   
4           True  C  T  ...         ELOVL fatty acid elongase 7   

   Overall_Gene_Score A1_in_CUD_AFR  Z_in_CUD_AFR  Z_in_CUD_AFR_corrected  \
0            0.066600             T         0.187              

/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2208053712.py:58: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_values[key] = binom_test(x=x1,n=n1, p=0.5, alternative='less')
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2208053712.py:68: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_values[key2] = binom_test(x=x2,n=n2, p=0.5, alternative='less')
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2208053712.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


In [130]:
w2 = ExcelWriter('fdr_out/condfdr_lead_snp2gene_effects_sign_concordance.xlsx')
for df in condfdr1.keys():
    df1 = condfdr1.get(df)
    tr1 = df.split('|')[0]
    tr2 = df.split('|')[1]
    if tr1 == 'AUD':
        smt1 = aud_smt1
    elif tr1 == 'CUD':
        smt1 = cud_smt2
    else:
        smt1 = oud_smt2
    if tr2 == 'AUD':
        smt2 = aud_smt1
    elif tr2 == 'CUD':
        smt2 = cud_smt2
    else:
        smt2 = oud_smt2
    df2 = df1.iloc[:, :-4]
    #print(df2.columns,tr1,smt1.columns,tr2,smt2.columns)
    #leadsnp4[f'A1_in_{trait1}'] = leadsnp4['CAND_SNP'].map(sm1.set_index('SNP')['A1'])
    smt1 = smt1.drop_duplicates('RSID')
    smt2 = smt2.drop_duplicates('RSID')
    df2[f'A1_in_{tr1}_AFR'] = df2['LEAD_SNP'].map(smt1.set_index('RSID')['A1'])
    df2[f'Z_in_{tr1}_AFR'] = df2['LEAD_SNP'].map(smt1.set_index('RSID')['Z'])
    df2[f'Z_in_{tr1}_AFR_corrected'] = np.where(df2['A1'] == df2[f'A1_in_{tr1}_AFR'],
                                                df2[f'Z_in_{tr1}_AFR'],df2[f'Z_in_{tr1}_AFR'] * -1)
    mask = df2[f'Z_in_{tr1}_AFR_corrected'].notna()

    # Applying conditional logic using np.where only on rows where Z_in_trait_AFR_corrected is not NaN
    df2.loc[mask, f'sign_concordance_{tr1}_AFR'] = np.where(
        ((df2.loc[mask, f'Z_in_{tr1}'] > 0) & (df2.loc[mask, f'Z_in_{tr1}_AFR_corrected'] > 0)) |
        ((df2.loc[mask, f'Z_in_{tr1}'] < 0) & (df2.loc[mask, f'Z_in_{tr1}_AFR_corrected'] < 0)),
        'Yes','No')
    
    df2[f'A1_in_{tr2}_AFR'] = df2['LEAD_SNP'].map(smt2.set_index('RSID')['A1'])
    df2[f'Z_in_{tr2}_AFR'] = df2['LEAD_SNP'].map(smt2.set_index('RSID')['Z'])
    df2[f'Z_in_{tr2}_AFR_corrected'] = np.where(df2['A1'] == df2[f'A1_in_{tr1}_AFR'],
                                                df2[f'Z_in_{tr1}_AFR'],df2[f'Z_in_{tr1}_AFR'] * -1)
    mask = df2[f'Z_in_{tr2}_AFR_corrected'].notna()

    # Applying conditional logic using np.where only on rows where Z_in_trait_AFR_corrected is not NaN
    df2.loc[mask, f'sign_concordance_{tr2}_AFR'] = np.where(
        ((df2.loc[mask, f'Z_in_{tr2}'] > 0) & (df2.loc[mask, f'Z_in_{tr2}_AFR_corrected'] > 0)) |
        ((df2.loc[mask, f'Z_in_{tr2}'] < 0) & (df2.loc[mask, f'Z_in_{tr2}_AFR_corrected'] < 0)),
        'Yes','No')
    
    # Create a dictionary to store p-values
    p_values = {}

    # Compute the number of "Yes" entries as successes
    x1 = df2.loc[mask, f'sign_concordance_{tr1}_AFR'].eq('No').sum()

    # Compute the total number of trials
    n1 = df2.loc[mask, f'sign_concordance_{tr1}_AFR'].count()

    # Compute the p-value and store it in the dictionary using dynamic key
    key = f'p_value_{tr1}'
    p_values[key] = binom_test(x=x1,n=n1, p=0.5, alternative='less')
    
    # Compute the number of "Yes" entries as successes
    x2 = df2.loc[mask, f'sign_concordance_{tr2}_AFR'].eq('No').sum()

    # Compute the total number of trials
    n2 = df2.loc[mask, f'sign_concordance_{tr2}_AFR'].count()

    # Compute the p-value and store it in the dictionary using dynamic key
    key2 = f'p_value_{tr2}'
    p_values[key2] = binom_test(x=x2,n=n2, p=0.5, alternative='less')
    results_df = results_df.append({
        "Trait1": tr1, 
        "P_value1": p_values[key],
        "Count_No1": x1,
        "Count_Total1": n1,
        "Trait2": tr2, 
        "P_value2": p_values[key2],
        "Count_No2": x2,
        "Count_Total2": n2 
    }, ignore_index=True)
    df2.to_excel(w2,sheet_name=f'{tr1}|{tr2}',index=False)
    results_df.to_excel(w2, sheet_name="binomial_summary", index=False)


    print(df2.head(),tr1,p_values[key],tr2,p_values[key2])
w2.close()

/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2852803761.py:58: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_values[key] = binom_test(x=x1,n=n1, p=0.5, alternative='less')
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2852803761.py:68: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_values[key2] = binom_test(x=x2,n=n2, p=0.5, alternative='less')
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2852803761.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


   locusnum  CHR    LEAD_SNP    LEAD_BP      MinBP      MaxBP       FDR  \
0         1    2  rs12616949   22594027   22549441   22606275  0.006116   
1         2    5  rs12513581   60533207   60484179   60614879  0.008724   
2         3    6    rs975303   19028788   19023726   19191459  0.001397   
3         4    6   rs3734540   26463321   26393021   26581168  0.001896   
4         5    7   rs1989903  114137940  114015707  114287116  0.001041   

   is_locus_lead A1 A2  ...                                     Approved name  \
0           True  A  G  ...                                               NaN   
1           True  C  T  ...                       ELOVL fatty acid elongase 7   
2           True  G  A  ...                                               NaN   
3           True  C  A  ...  high mobility group nucleosomal binding domain 4   
4           True  A  G  ...                                   forkhead box P2   

   Overall_Gene_Score A1_in_CUD_AFR  Z_in_CUD_AFR  Z_in_CUD_AF

/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2852803761.py:58: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_values[key] = binom_test(x=x1,n=n1, p=0.5, alternative='less')
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2852803761.py:68: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_values[key2] = binom_test(x=x2,n=n2, p=0.5, alternative='less')
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2852803761.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


   locusnum  CHR    LEAD_SNP   LEAD_BP     MinBP     MaxBP       FDR  \
0         1    1   rs1392816  66481188  66334537  66485442  0.000036   
1         2    1  rs12122743  91220791  91200757  91234126  0.006585   
2         3    5    rs173946  60488297  60135962  60550041  0.001336   
3         4    5   rs6452787  87712831  87514778  87822672  0.003577   
4         5    6  rs72828558  19128832  19003340  19191459  0.003909   

   is_locus_lead A1 A2  ...                           Approved name  \
0           True  T  C  ...  leptin receptor overlapping transcript   
1           True  G  A  ...                    BarH like homeobox 2   
2           True  C  T  ...             ELOVL fatty acid elongase 7   
3           True  G  A  ...              transmembrane protein 161B   
4           True  C  T  ...                                     NaN   

   Overall_Gene_Score A1_in_CUD_AFR  Z_in_CUD_AFR  Z_in_CUD_AFR_corrected  \
0            0.079678             C         0.579              

/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2852803761.py:58: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_values[key] = binom_test(x=x1,n=n1, p=0.5, alternative='less')
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2852803761.py:68: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_values[key2] = binom_test(x=x2,n=n2, p=0.5, alternative='less')
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2852803761.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


...

/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2852803761.py:58: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_values[key] = binom_test(x=x1,n=n1, p=0.5, alternative='less')
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2852803761.py:68: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_values[key2] = binom_test(x=x2,n=n2, p=0.5, alternative='less')
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2852803761.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


   locusnum  CHR    LEAD_SNP    LEAD_BP      MinBP      MaxBP           FDR  \
0         1    1  rs12031155   53714139   53658317   53752134  9.997570e-03   
1         2    2  rs35942385  144208523  144145478  144263280  2.498582e-03   
2         3    6  rs72828517   19036035   19023726   19191459  7.457391e-03   
3         4    6   rs3094551   29354799   28674322   29354809  2.603033e-04   
4         5    6   rs1799971  154360797  154194081  154390607  2.066166e-08   

   is_locus_lead A1 A2  ...                     Approved name  \
0           True  T  C  ...  carnitine palmitoyltransferase 2   
1           True  T  G  ...  Rho GTPase activating protein 15   
2           True  C  T  ...          ring finger protein 144B   
3           True  A  G  ...         ZFP57 zinc finger protein   
4           True  G  A  ...              opioid receptor mu 1   

   Overall_Gene_Score A1_in_OUD_AFR  Z_in_OUD_AFR  Z_in_OUD_AFR_corrected  \
0            0.213481             T    -17.756744        

/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2852803761.py:58: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_values[key] = binom_test(x=x1,n=n1, p=0.5, alternative='less')
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2852803761.py:68: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_values[key2] = binom_test(x=x2,n=n2, p=0.5, alternative='less')
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2852803761.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


   locusnum  CHR    LEAD_SNP    LEAD_BP      MinBP      MaxBP       FDR  \
0         1    1   rs2310819   66440096   66334518   66547212  0.000001   
1         2    1  rs12145432   73769330   73766431   73991651  0.000226   
2         3    1  rs61535100   80827756   80784642   80871734  0.004921   
3         4    1   rs2166171   91208451   91189731   91234126  0.000117   
4         5    1   rs7544174  236902955  236897248  236905585  0.003473   

   is_locus_lead A1 A2  ...                           Approved name  \
0           True  C  T  ...  leptin receptor overlapping transcript   
1           True  T  G  ...                                     NaN   
2           True  G  A  ...                                     NaN   
3           True  C  T  ...                    BarH like homeobox 2   
4           True  T  C  ...                         actinin alpha 2   

   Overall_Gene_Score A1_in_AUD_AFR  Z_in_AUD_AFR  Z_in_AUD_AFR_corrected  \
0            0.066398             T         0

/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2852803761.py:58: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_values[key] = binom_test(x=x1,n=n1, p=0.5, alternative='less')
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2852803761.py:68: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_values[key2] = binom_test(x=x2,n=n2, p=0.5, alternative='less')
/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/2852803761.py:69: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


In [121]:
binom_test(x=0,n=5, p=0.5, alternative='less')

/var/folders/g7/p8558cx50s5gdqg8zvqrdf1h0000gn/T/ipykernel_1874/577880601.py:1: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  binom_test(x=0,n=5, p=0.5, alternative='less')


0.03125